In [17]:
from optimum.intel.openvino import OVStableDiffusionPipeline
from IPython.display import Image, display

In [18]:
q_ov_pipe = OVStableDiffusionPipeline.from_pretrained("sd-model-quantized/openvino", compile=False)
q_ov_pipe.reshape(batch_size=1, height=512, width=512, num_images_per_prompt=1)
q_ov_pipe.compile()

print("Done")

The config attributes {'requires_safety_checker': True, 'safety_checker': [None, None]} were passed to OVStableDiffusionPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Compiling the encoder and creating the inference request ...
Compiling the encoder and creating the inference request ...
Compiling the encoder and creating the inference request ...


Done


In [20]:
prompt = "Super cute fluffy cat warrior in armor, photorealistic, 4K, ultra detailed, vray rendering, unreal engine"#"plant pokemon in jungle"#"a drawing of a green pokemon with red eyes"#"sailing ship in storm by Rembrandt"#
output = q_ov_pipe(prompt, num_inference_steps=50, output_type="pil")
display(output.images[0])

  0%|          | 0/50 [00:00<?, ?it/s]